# Code

In [2]:
import pandas as pd
import numpy as np
import math
from copy import deepcopy
from random import randint

class KMeans:
    __valid_distance_method = ['manhattan', 'euclidian']
    
    def __init__(self, n_clusters = 2, max_iter = 300, distance_method = 'euclidian'):
        if (distance_method not in self.__valid_distance_method):
            raise ValueError("Unknown distance calculation type type %s."
                             "Valid options are %s" % (distance,
                                                       self.__valid_distance_method))
        if (n_clusters < 1):
            raise ValueError("Invalid value of n_clusters, got %d. Must be at least 1" % n_clusters)
        self.__n_clusters = n_clusters
        self.__max_iter = max_iter
        self.__distance_method = distance_method
        
    def fit(self, data):
        if (len(data) < 2):
            raise ValueError("Need at least 2 instance of data")
        # TODO: check type(data)
        self.__data = data.values.tolist()
        self.__n_data = len(data)
        self.__attr = data.columns.tolist()
        self.__n_attr = len(self.__attr)
        self.labels_ = np.array([])
        self.cluster_centers_ = []
        self.initial_cluster_centers_idx_ = []
        
        # randomly select k instance for initial cluster
        for i in range(self.__n_clusters):
            idx = randint(0, self.__n_data)
            while (idx in self.initial_cluster_centers_idx_):
                idx = randint(0, self.__n_data)
            self.initial_cluster_centers_idx_.append(idx)
            self.cluster_centers_.append(self.__data[idx])
            
        self.__is_convergence = False
        self.__clusters_items = []
        iter_count = 0
        while (not self.__is_convergence) and iter_count < self.__max_iter:
            self.labels_ = []
            self.__clusters_items = [[] for _ in xrange(self.__n_clusters)]
            
            # iterate through all data, and calculate its distance with current centroid
            for i in range(self.__n_data):
                clusters_distance = []
                for center in self.cluster_centers_:
                    clusters_distance.append(
                        self.__calculate_distance(self.__data[i], center, self.__distance_method)
                    )

                cluster = np.argmin(clusters_distance)
                self.labels_.append(cluster)
                self.__clusters_items[cluster].append(self.__data[i])
                

            old_cluster_centers = deepcopy(self.cluster_centers_)
            # calculate new means per cluster
            for i in range(self.__n_clusters):
                cluster_attr_sum = np.array([0] * self.__n_attr)
                n_member = len(self.__clusters_items[i])
                
                if n_member == 0:
                    self.cluster_centers_[i] = self.__data[randint(0, self.__n_data)]
                    continue
                
                for item in self.__clusters_items[i]:
                    for j, attr in enumerate(item):
                        cluster_attr_sum[j] += attr
                
                for j in range(self.__n_attr):
                    cluster_attr_sum[j] = float(float(cluster_attr_sum[j]) / float(n_member))
            
            
            # check if convergence
            old = [item for sublist in old_cluster_centers for item in sublist]
            new = [item for sublist in self.cluster_centers_ for item in sublist]
            if (old[1:] == new[:-1]):
                self.__is_convergence = True
                    
            iter_count += 1
            
        return self
    
    def __calculate_distance(self, X, Y, method = 'euclidian'):
        if (len(X) != len (Y)):
            raise ValueError("Can't calculate distance with across dimension")
        
        if method == 'manhattan':
            return self.__calculate_distance_manhattan(X,Y)
        
        return self.__calculate_distance_euclidian(X,Y)
            
    def __calculate_distance_euclidian(self, X, Y):
        square_sum = 0
        for i in range(len(X)):
            square_sum += (X[i]-Y[i]) ** 2
        return math.sqrt(square_sum)
    
    def __calculate_distance_manhattan(self, X, Y):
        distance_sum = 0;
        for i in range(len(X)):
            distance_sum += abs(X[i]-Y[i])
        return distance_sum;
        

# Contoh pemakaian

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

iris = pd.read_csv('Iris.csv')
iris = iris.drop(labels='Id', axis=1)

iris_data = iris.iloc[:,:-1]
iris_label = iris.iloc[:,-1]

In [4]:
iris_data.columns.tolist()

['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

In [7]:
for _ in range(5):
    kmeans = KMeans(n_clusters = 3, distance_method = 'euclidian').fit(iris_data)
    labels = kmeans.labels_

In [8]:
pd.crosstab(iris_label, np.array(kmeans.labels_))

col_0,0,1,2
Species,,,
Iris-setosa,0,0,50
Iris-versicolor,3,47,0
Iris-virginica,40,10,0


In [6]:
kmeans.cluster_centers_

[[6.2, 2.2, 4.5, 1.5], [5.0, 3.4, 1.6, 0.4], [6.1, 2.8, 4.0, 1.3]]

Perbandingan dengan SKLearn

In [7]:
from sklearn.cluster import KMeans as km

xx = km(init='random', n_clusters=3).fit(iris_data)

pd.crosstab(iris_label, xx.labels_)

col_0,0,1,2
Species,,,
Iris-setosa,50,0,0
Iris-versicolor,0,48,2
Iris-virginica,0,14,36
